In [14]:
import sys
import subprocess
import re
from pathlib import Path

subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "django", "pillow"])

print("OK: Django + Pillow установлены")

OK: Django + Pillow установлены


In [15]:
BASE_DIR = Path(".").resolve()
PROJECT_NAME = "myblog"
APP_NAME = "blog"

project_dir = BASE_DIR / PROJECT_NAME
manage_py = project_dir / "manage.py"

if not manage_py.exists():
    subprocess.check_call([sys.executable, "-m", "django", "startproject", PROJECT_NAME])
    print("OK: project created ->", project_dir)
else:
    print("SKIP: project already exists ->", project_dir)

app_dir = project_dir / APP_NAME
if not app_dir.exists():
    subprocess.check_call([sys.executable, "manage.py", "startapp", APP_NAME], cwd=str(project_dir))
    print("OK: app created ->", app_dir)
else:
    print("SKIP: app already exists ->", app_dir)

print("Готово, блоки внутри myblog.")


SKIP: project already exists -> /Users/dmitry/myblog
SKIP: app already exists -> /Users/dmitry/myblog/blog
Готово, блоки внутри myblog.


In [18]:
PROJECT_DIR = Path("myblog").resolve()
manage_py = PROJECT_DIR / "manage.py"
if not manage_py.exists():
    raise FileNotFoundError("Не найден myblog/manage.py. Сначала запусти БЛОК 2.")

settings_py = None
for p in PROJECT_DIR.rglob("settings.py"):
    settings_py = p
    break

txt = settings_py.read_text(encoding="utf-8")

if re.search(r"INSTALLED_APPS\s*=\s*\[", txt) and not re.search(r"['\"]blog['\"]", txt):
    txt = re.sub(
        r"(INSTALLED_APPS\s*=\s*\[\s*)([\s\S]*?)(\])",
        lambda m: m.group(1) + m.group(2).rstrip() + "\n    'blog',\n" + m.group(3),
        txt,
        count=1
    )
    print("OK: 'blog' добавлен в INSTALLED_APPS")
else:
    print("SKIP: 'blog' уже есть в INSTALLED_APPS")

marker = "# === MEDIA SETTINGS (blog) ==="
if marker not in txt:
    block = f"""
{marker}
import os
MEDIA_URL = '/media/'
MEDIA_ROOT = os.path.join(BASE_DIR, 'media')
# === END MEDIA SETTINGS (blog) ===
""".strip()
    txt = txt.rstrip() + "\n\n" + block + "\n"
    print("OK: MEDIA_URL/MEDIA_ROOT добавлены")
else:
    print("SKIP: MEDIA_URL/MEDIA_ROOT уже добавлены")

settings_py.write_text(txt, encoding="utf-8")
print("OK: updated ->", settings_py)


SKIP: 'blog' уже есть в INSTALLED_APPS
SKIP: MEDIA_URL/MEDIA_ROOT уже добавлены
OK: updated -> /Users/dmitry/myblog/myblog/settings.py


In [19]:
PROJECT_DIR = Path("myblog").resolve()
models_py = PROJECT_DIR / "blog" / "models.py"

models_content = """\
from django.db import models
from django.utils import timezone
from django.contrib.auth.models import User


class Category(models.Model):
    name = models.CharField(max_length=100, verbose_name='Название категории')
    description = models.TextField(blank=True, verbose_name='Описание')

    def __str__(self):
        return self.name

    class Meta:
        verbose_name = 'Категория'
        verbose_name_plural = 'Категории'


class Post(models.Model):
    title = models.CharField(max_length=200, verbose_name='Заголовок')
    content = models.TextField(verbose_name='Содержание')
    created_date = models.DateTimeField(default=timezone.now, verbose_name='Дата создания')
    published_date = models.DateTimeField(blank=True, null=True, verbose_name='Дата публикации')
    author = models.ForeignKey(User, on_delete=models.CASCADE, verbose_name='Автор')
    category = models.ForeignKey(Category, on_delete=models.SET_NULL, null=True, verbose_name='Категория')
    image = models.ImageField(upload_to='post_images/', blank=True, null=True, verbose_name='Изображение')

    def publish(self):
        self.published_date = timezone.now()
        self.save()

    def __str__(self):
        return self.title

    class Meta:
        verbose_name = 'Пост'
        verbose_name_plural = 'Посты'
"""
models_py.write_text(models_content, encoding="utf-8")
print("OK: wrote ->", models_py)


OK: wrote -> /Users/dmitry/myblog/blog/models.py


In [25]:
PROJECT_DIR = Path("myblog").resolve()

subprocess.check_call([sys.executable, "manage.py", "makemigrations", "blog"], cwd=str(PROJECT_DIR))
subprocess.check_call([sys.executable, "manage.py", "migrate"], cwd=str(PROJECT_DIR))

print("OK: blog migrations done")


No changes detected in app 'blog'
Operations to perform:
  Apply all migrations: admin, auth, blog, contenttypes, sessions
Running migrations:
  No migrations to apply.
OK: blog migrations done


In [26]:
PROJECT_DIR = Path("myblog").resolve()
views_py = PROJECT_DIR / "blog" / "views.py"

views_content = """\
from django.shortcuts import render, get_object_or_404
from django.utils import timezone
from .models import Post, Category


def post_list(request):
    posts = Post.objects.filter(published_date__lte=timezone.now()).order_by('-published_date')
    return render(request, 'blog/post_list.html', {'posts': posts})


def post_detail(request, pk):
    post = get_object_or_404(Post, pk=pk)
    return render(request, 'blog/post_detail.html', {'post': post})


def category_posts(request, category_id):
    category = get_object_or_404(Category, id=category_id)
    posts = Post.objects.filter(category=category, published_date__lte=timezone.now()).order_by('-published_date')
    return render(request, 'blog/category_posts.html', {'category': category, 'posts': posts})
"""
views_py.write_text(views_content, encoding="utf-8")
print("OK: wrote ->", views_py)


OK: wrote -> /Users/dmitry/myblog/blog/views.py


In [27]:
PROJECT_DIR = Path("myblog").resolve()

blog_urls_py = PROJECT_DIR / "blog" / "urls.py"
blog_urls_py.write_text("""\
from django.urls import path
from . import views

urlpatterns = [
    path('', views.post_list, name='post_list'),
    path('post/<int:pk>/', views.post_detail, name='post_detail'),
    path('category/<int:category_id>/', views.category_posts, name='category_posts'),
]
""", encoding="utf-8")
print("OK: wrote ->", blog_urls_py)

project_urls_py = PROJECT_DIR / "myblog" / "urls.py"
project_urls_py.write_text("""\
from django.contrib import admin
from django.urls import path, include
from django.conf import settings
from django.conf.urls.static import static

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('blog.urls')),
]

if settings.DEBUG:
    urlpatterns += static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)
""", encoding="utf-8")
print("OK: wrote ->", project_urls_py)


OK: wrote -> /Users/dmitry/myblog/blog/urls.py
OK: wrote -> /Users/dmitry/myblog/myblog/urls.py


In [28]:
PROJECT_DIR = Path("myblog").resolve()
tpl_dir = PROJECT_DIR / "blog" / "templates" / "blog"
tpl_dir.mkdir(parents=True, exist_ok=True)

(tpl_dir / "base.html").write_text("""\
<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}Мой Блог{% endblock %}</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 0; padding: 0; }
        .header { background: #333; color: white; padding: 1rem; }
        .nav { background: #f4f4f4; padding: 1rem; }
        .content { padding: 2rem; }
        .post { border: 1px solid #ddd; margin-bottom: 1rem; padding: 1rem; }
        .post img { max-width: 200px; }
    </style>
</head>
<body>
    <div class="header">
        <h1><a href="/" style="color: white; text-decoration: none;">Мой Персональный Блог</a></h1>
    </div>

    <div class="nav">
        <a href="/">Главная</a> |
        <a href="/admin/">Админ-панель</a>
    </div>

    <div class="content">
        {% block content %}
        {% endblock %}
    </div>
</body>
</html>
""", encoding="utf-8")

(tpl_dir / "post_list.html").write_text("""\
{% extends 'blog/base.html' %}

{% block title %}Главная страница{% endblock %}

{% block content %}
<h2>Последние посты</h2>

{% for post in posts %}
    <div class="post">
        <h3><a href="{% url 'post_detail' pk=post.pk %}">{{ post.title }}</a></h3>
        {% if post.image %}
            <img src="{{ post.image.url }}" alt="{{ post.title }}">
        {% endif %}
        <p>{{ post.content|truncatewords:30 }}</p>
        <small>
            Автор: {{ post.author }} |
            Опубликовано: {{ post.published_date|date:"d.m.Y H:i" }} |
            Категория: {{ post.category.name }}
        </small>
    </div>
{% empty %}
    <p>Пока нет постов.</p>
{% endfor %}
{% endblock %}
""", encoding="utf-8")

(tpl_dir / "post_detail.html").write_text("""\
{% extends 'blog/base.html' %}

{% block title %}{{ post.title }}{% endblock %}

{% block content %}
    <article class="post">
        <h2>{{ post.title }}</h2>
        {% if post.image %}
            <img src="{{ post.image.url }}" alt="{{ post.title }}">
        {% endif %}
        <p>{{ post.content|linebreaks }}</p>
        <div class="post-meta">
            <p><strong>Автор:</strong> {{ post.author }}</p>
            <p><strong>Опубликовано:</strong> {{ post.published_date|date:"d.m.Y H:i" }}</p>
            <p><strong>Категория:</strong> {{ post.category.name }}</p>
        </div>
        <a href="{% url 'post_list' %}">← Назад к списку постов</a>
    </article>
{% endblock %}
""", encoding="utf-8")

(tpl_dir / "category_posts.html").write_text("""\
{% extends 'blog/base.html' %}

{% block title %}Категория: {{ category.name }}{% endblock %}

{% block content %}
<h2>Категория: {{ category.name }}</h2>
<p>{{ category.description }}</p>

{% for post in posts %}
    <div class="post">
        <h3><a href="{% url 'post_detail' pk=post.pk %}">{{ post.title }}</a></h3>
        <p>{{ post.content|truncatewords:30 }}</p>
        <small>Опубликовано: {{ post.published_date|date:"d.m.Y H:i" }}</small>
    </div>
{% empty %}
    <p>В этой категории пока нет постов.</p>
{% endfor %}
{% endblock %}
""", encoding="utf-8")

print("OK: templates written ->", tpl_dir)


OK: templates written -> /Users/dmitry/myblog/blog/templates/blog


In [29]:
PROJECT_DIR = Path("myblog").resolve()
cmd_dir = PROJECT_DIR / "blog" / "management" / "commands"
cmd_dir.mkdir(parents=True, exist_ok=True)

fill_db_py = cmd_dir / "fill_db.py"
fill_db_py.write_text("""\
from django.core.management.base import BaseCommand
from blog.models import Category, Post
from django.contrib.auth.models import User


class Command(BaseCommand):
    help = 'Заполнение базы данных тестовыми данными'

    def handle(self, *args, **options):
        user, created = User.objects.get_or_create(
            username='testuser',
            defaults={'email': 'test@example.com'}
        )
        if created:
            user.set_password('testpassword123')
            user.save()

        categories_data = [
            {'name': 'Программирование', 'description': 'Статьи о программировании'},
            {'name': 'Путешествия', 'description': 'Рассказы о путешествиях'},
            {'name': 'Кулинария', 'description': 'Рецепты и советы по готовке'},
        ]

        for cat_data in categories_data:
            category, created = Category.objects.get_or_create(**cat_data)
            if created:
                self.stdout.write(f'Создана категория: {category.name}')

        posts_data = [
            {
                'title': 'Мой первый пост в блоге',
                'content': 'Это содержимое моего первого поста. Я только начинаю вести блог и хочу делиться своими мыслями и идеями.',
                'author': user,
                'category': Category.objects.get(name='Программирование')
            },
            {
                'title': 'Лучшие места для путешествий',
                'content': 'В этом посте я расскажу о самых красивых местах, которые я посетил за последний год.',
                'author': user,
                'category': Category.objects.get(name='Путешествия')
            },
            {
                'title': 'Простой рецепт пасты',
                'content': 'Сегодня поделюсь с вами своим любимым рецептом пасты карбонара.',
                'author': user,
                'category': Category.objects.get(name='Кулинария')
            },
        ]

        for post_data in posts_data:
            post = Post.objects.create(**post_data)
            post.publish()
            self.stdout.write(f'Создан пост: {post.title}')

        self.stdout.write(self.style.SUCCESS('База данных успешно заполнена!'))
""", encoding="utf-8")

print("OK: wrote ->", fill_db_py)

subprocess.check_call([sys.executable, "manage.py", "fill_db"], cwd=str(PROJECT_DIR))
print("OK: fill_db executed")


OK: wrote -> /Users/dmitry/myblog/blog/management/commands/fill_db.py
Создана категория: Программирование
Создана категория: Путешествия
Создана категория: Кулинария
Создан пост: Мой первый пост в блоге
Создан пост: Лучшие места для путешествий
Создан пост: Простой рецепт пасты
База данных успешно заполнена!
OK: fill_db executed


In [41]:
PROJECT_DIR = Path("myblog").resolve()

if "server_proc" in globals() and server_proc and server_proc.poll() is None:
    server_proc.terminate()

server_proc = subprocess.Popen(
    [sys.executable, "manage.py", "runserver", "--noreload", "127.0.0.1:8001"],
    cwd=str(PROJECT_DIR),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

print("OK: открой http://127.0.0.1:8001/")


OK: открой http://127.0.0.1:8001/


In [47]:
# ДОП ЗАДАНИЕ 1

PROJECT_DIR = Path("myblog").resolve()

views_py = PROJECT_DIR / "blog" / "views.py"
views_py.write_text("""\
from django.shortcuts import render, get_object_or_404
from django.utils import timezone
from django.db.models import Q
from .models import Post, Category


def post_list(request):
    q = request.GET.get("q", "").strip()
    posts = Post.objects.filter(published_date__lte=timezone.now()).order_by('-published_date')

    if q:
        posts = posts.filter(Q(title__icontains=q) | Q(content__icontains=q))

    return render(request, 'blog/post_list.html', {'posts': posts, 'q': q})


def post_detail(request, pk):
    post = get_object_or_404(Post, pk=pk)
    return render(request, 'blog/post_detail.html', {'post': post})


def category_posts(request, category_id):
    category = get_object_or_404(Category, id=category_id)
    q = request.GET.get("q", "").strip()
    posts = Post.objects.filter(category=category, published_date__lte=timezone.now()).order_by('-published_date')

    if q:
        posts = posts.filter(Q(title__icontains=q) | Q(content__icontains=q))

    return render(request, 'blog/category_posts.html', {'category': category, 'posts': posts, 'q': q})
""", encoding="utf-8")

tpl_dir = PROJECT_DIR / "blog" / "templates" / "blog"
base_py = tpl_dir / "base.html"
base_py.write_text("""\
<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}Мой Блог{% endblock %}</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 0; padding: 0; }
        .header { background: #333; color: white; padding: 1rem; }
        .nav { background: #f4f4f4; padding: 1rem; display:flex; gap:16px; align-items:center; flex-wrap:wrap; }
        .content { padding: 2rem; }
        .post { border: 1px solid #ddd; margin-bottom: 1rem; padding: 1rem; }
        .post img { max-width: 200px; }
        input[type="text"] { padding:6px; }
        button { padding:6px 10px; }
    </style>
</head>
<body>
    <div class="header">
        <h1><a href="/" style="color: white; text-decoration: none;">Мой Персональный Блог</a></h1>
    </div>

    <div class="nav">
        <a href="/">Главная</a>
        <a href="/admin/">Админ-панель</a>

        <form method="get" action="/" style="margin-left:auto; display:flex; gap:8px; align-items:center;">
            <input type="text" name="q" value="{{ q|default:'' }}" placeholder="Поиск по постам..." />
            <button type="submit">Найти</button>
        </form>
    </div>

    <div class="content">
        {% block content %}{% endblock %}
    </div>
</body>
</html>
""", encoding="utf-8")

post_list_py = tpl_dir / "post_list.html"
post_list_py.write_text("""\
{% extends 'blog/base.html' %}

{% block title %}Главная страница{% endblock %}

{% block content %}
<h2>Последние посты</h2>

{% if q %}
    <p>Результаты по запросу: <strong>{{ q }}</strong></p>
{% endif %}

{% for post in posts %}
    <div class="post">
        <h3><a href="{% url 'post_detail' pk=post.pk %}">{{ post.title }}</a></h3>
        {% if post.image %}
            <img src="{{ post.image.url }}" alt="{{ post.title }}">
        {% endif %}
        <p>{{ post.content|truncatewords:30 }}</p>
        <small>
            Автор: {{ post.author }} |
            Опубликовано: {{ post.published_date|date:"d.m.Y H:i" }} |
            Категория: {{ post.category.name }}
        </small>
    </div>
{% empty %}
    <p>Ничего не найдено.</p>
{% endfor %}
{% endblock %}
""", encoding="utf-8")

category_py = tpl_dir / "category_posts.html"
category_py.write_text("""\
{% extends 'blog/base.html' %}

{% block title %}Категория: {{ category.name }}{% endblock %}

{% block content %}
<h2>Категория: {{ category.name }}</h2>
<p>{{ category.description }}</p>

{% if q %}
    <p>Результаты по запросу: <strong>{{ q }}</strong></p>
{% endif %}

{% for post in posts %}
    <div class="post">
        <h3><a href="{% url 'post_detail' pk=post.pk %}">{{ post.title }}</a></h3>
        <p>{{ post.content|truncatewords:30 }}</p>
        <small>Опубликовано: {{ post.published_date|date:"d.m.Y H:i" }}</small>
    </div>
{% empty %}
    <p>Ничего не найдено.</p>
{% endfor %}
{% endblock %}
""", encoding="utf-8")

print("OK: search added (views + templates)")


OK: search added (views + templates)


In [48]:
# ДОП ЗАДАНИЕ 2

PROJECT_DIR = Path("myblog").resolve()

models_py = PROJECT_DIR / "blog" / "models.py"
models_py.write_text("""\
from django.db import models
from django.utils import timezone
from django.contrib.auth.models import User


class Category(models.Model):
    name = models.CharField(max_length=100, verbose_name='Название категории')
    description = models.TextField(blank=True, verbose_name='Описание')

    def __str__(self):
        return self.name

    class Meta:
        verbose_name = 'Категория'
        verbose_name_plural = 'Категории'


class Post(models.Model):
    title = models.CharField(max_length=200, verbose_name='Заголовок')
    content = models.TextField(verbose_name='Содержание')
    created_date = models.DateTimeField(default=timezone.now, verbose_name='Дата создания')
    published_date = models.DateTimeField(blank=True, null=True, verbose_name='Дата публикации')
    author = models.ForeignKey(User, on_delete=models.CASCADE, verbose_name='Автор')
    category = models.ForeignKey(Category, on_delete=models.SET_NULL, null=True, verbose_name='Категория')
    image = models.ImageField(upload_to='post_images/', blank=True, null=True, verbose_name='Изображение')

    def publish(self):
        self.published_date = timezone.now()
        self.save()

    def __str__(self):
        return self.title

    class Meta:
        verbose_name = 'Пост'
        verbose_name_plural = 'Посты'


class Comment(models.Model):
    post = models.ForeignKey(Post, on_delete=models.CASCADE, related_name='comments', verbose_name='Пост')
    name = models.CharField(max_length=80, verbose_name='Имя')
    email = models.EmailField(blank=True, verbose_name='Email')
    text = models.TextField(verbose_name='Комментарий')
    created_date = models.DateTimeField(default=timezone.now, verbose_name='Дата')
    is_approved = models.BooleanField(default=True, verbose_name='Одобрен')

    def __str__(self):
        return f"{self.name}: {self.text[:30]}"

    class Meta:
        verbose_name = 'Комментарий'
        verbose_name_plural = 'Комментарии'
""", encoding="utf-8")

forms_py = PROJECT_DIR / "blog" / "forms.py"
forms_py.write_text("""\
from django import forms
from .models import Comment


class CommentForm(forms.ModelForm):
    class Meta:
        model = Comment
        fields = ['name', 'email', 'text']
""", encoding="utf-8")

views_py = PROJECT_DIR / "blog" / "views.py"
views_py.write_text("""\
from django.shortcuts import render, get_object_or_404, redirect
from django.utils import timezone
from django.db.models import Q
from .models import Post, Category
from .forms import CommentForm


def post_list(request):
    q = request.GET.get("q", "").strip()
    posts = Post.objects.filter(published_date__lte=timezone.now()).order_by('-published_date')
    if q:
        posts = posts.filter(Q(title__icontains=q) | Q(content__icontains=q))
    return render(request, 'blog/post_list.html', {'posts': posts, 'q': q})


def post_detail(request, pk):
    post = get_object_or_404(Post, pk=pk)
    comments = post.comments.filter(is_approved=True).order_by('created_date')

    if request.method == 'POST':
        form = CommentForm(request.POST)
        if form.is_valid():
            c = form.save(commit=False)
            c.post = post
            c.save()
            return redirect('post_detail', pk=post.pk)
    else:
        form = CommentForm()

    return render(request, 'blog/post_detail.html', {'post': post, 'comments': comments, 'form': form})


def category_posts(request, category_id):
    category = get_object_or_404(Category, id=category_id)
    q = request.GET.get("q", "").strip()
    posts = Post.objects.filter(category=category, published_date__lte=timezone.now()).order_by('-published_date')
    if q:
        posts = posts.filter(Q(title__icontains=q) | Q(content__icontains=q))
    return render(request, 'blog/category_posts.html', {'category': category, 'posts': posts, 'q': q})
""", encoding="utf-8")

tpl_dir = PROJECT_DIR / "blog" / "templates" / "blog"
post_detail_py = tpl_dir / "post_detail.html"
post_detail_py.write_text("""\
{% extends 'blog/base.html' %}

{% block title %}{{ post.title }}{% endblock %}

{% block content %}
<article class="post">
    <h2>{{ post.title }}</h2>
    {% if post.image %}
        <img src="{{ post.image.url }}" alt="{{ post.title }}">
    {% endif %}

    <p>{{ post.content|linebreaks }}</p>

    <div class="post-meta">
        <p><strong>Автор:</strong> {{ post.author }}</p>
        <p><strong>Опубликовано:</strong> {{ post.published_date|date:"d.m.Y H:i" }}</p>
        <p><strong>Категория:</strong> {{ post.category.name }}</p>
    </div>

    <hr>

    <h3>Комментарии</h3>
    {% for c in comments %}
        <div style="border:1px solid #ddd; padding:10px; margin-bottom:10px;">
            <strong>{{ c.name }}</strong>
            <small style="margin-left:10px;">{{ c.created_date|date:"d.m.Y H:i" }}</small>
            <p>{{ c.text|linebreaks }}</p>
        </div>
    {% empty %}
        <p>Комментариев пока нет.</p>
    {% endfor %}

    <h3>Оставить комментарий</h3>
    <form method="post">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit">Отправить</button>
    </form>

    <br>
    <a href="{% url 'post_list' %}">← Назад к списку постов</a>
</article>
{% endblock %}
""", encoding="utf-8")

admin_py = PROJECT_DIR / "blog" / "admin.py"
admin_py.write_text("""\
from django.contrib import admin
from .models import Category, Post, Comment


@admin.register(Category)
class CategoryAdmin(admin.ModelAdmin):
    list_display = ['name', 'description']
    search_fields = ['name']


@admin.register(Post)
class PostAdmin(admin.ModelAdmin):
    list_display = ['title', 'author', 'created_date', 'published_date', 'category']
    list_filter = ['created_date', 'published_date', 'category']
    search_fields = ['title', 'content']
    date_hierarchy = 'created_date'

    fieldsets = (
        ('Основная информация', {'fields': ('title', 'content', 'author', 'category')}),
        ('Даты', {'fields': ('created_date', 'published_date')}),
        ('Медиа', {'fields': ('image',)}),
    )


@admin.register(Comment)
class CommentAdmin(admin.ModelAdmin):
    list_display = ['post', 'name', 'created_date', 'is_approved']
    list_filter = ['is_approved', 'created_date']
    search_fields = ['name', 'email', 'text']
""", encoding="utf-8")

subprocess.check_call([sys.executable, "manage.py", "makemigrations", "blog"], cwd=str(PROJECT_DIR))
subprocess.check_call([sys.executable, "manage.py", "migrate"], cwd=str(PROJECT_DIR))

print("OK: comments added + migrated")


Migrations for 'blog':
  blog/migrations/0002_comment.py
    + Create model Comment
Operations to perform:
  Apply all migrations: admin, auth, blog, contenttypes, sessions
Running migrations:
  Applying blog.0002_comment... OK
OK: comments added + migrated


In [50]:
# ДОП ЗАДАНИЕ 3

PROJECT_DIR = Path("myblog").resolve()

views_py = PROJECT_DIR / "blog" / "views.py"
views_py.write_text("""\
from django.shortcuts import render, get_object_or_404, redirect
from django.utils import timezone
from django.db.models import Q
from django.core.paginator import Paginator

from .models import Post, Category
from .forms import CommentForm


def post_list(request):
    q = request.GET.get("q", "").strip()
    qs = Post.objects.filter(published_date__lte=timezone.now()).order_by('-published_date')
    if q:
        qs = qs.filter(Q(title__icontains=q) | Q(content__icontains=q))

    paginator = Paginator(qs, 5)
    page_number = request.GET.get("page")
    page_obj = paginator.get_page(page_number)

    return render(request, 'blog/post_list.html', {'page_obj': page_obj, 'q': q})


def post_detail(request, pk):
    post = get_object_or_404(Post, pk=pk)
    comments = post.comments.filter(is_approved=True).order_by('created_date')

    if request.method == 'POST':
        form = CommentForm(request.POST)
        if form.is_valid():
            c = form.save(commit=False)
            c.post = post
            c.save()
            return redirect('post_detail', pk=post.pk)
    else:
        form = CommentForm()

    return render(request, 'blog/post_detail.html', {'post': post, 'comments': comments, 'form': form})


def category_posts(request, category_id):
    category = get_object_or_404(Category, id=category_id)
    q = request.GET.get("q", "").strip()
    qs = Post.objects.filter(category=category, published_date__lte=timezone.now()).order_by('-published_date')
    if q:
        qs = qs.filter(Q(title__icontains=q) | Q(content__icontains=q))

    paginator = Paginator(qs, 5)
    page_number = request.GET.get("page")
    page_obj = paginator.get_page(page_number)

    return render(request, 'blog/category_posts.html', {'category': category, 'page_obj': page_obj, 'q': q})
""", encoding="utf-8")

tpl_dir = PROJECT_DIR / "blog" / "templates" / "blog"

# post_list.html (page_obj + навигация)
(tpl_dir / "post_list.html").write_text("""\
{% extends 'blog/base.html' %}

{% block title %}Главная страница{% endblock %}

{% block content %}
<h2>Последние посты</h2>

{% if q %}
    <p>Результаты по запросу: <strong>{{ q }}</strong></p>
{% endif %}

{% for post in page_obj %}
    <div class="post">
        <h3><a href="{% url 'post_detail' pk=post.pk %}">{{ post.title }}</a></h3>
        {% if post.image %}
            <img src="{{ post.image.url }}" alt="{{ post.title }}">
        {% endif %}
        <p>{{ post.content|truncatewords:30 }}</p>
        <small>
            Автор: {{ post.author }} |
            Опубликовано: {{ post.published_date|date:"d.m.Y H:i" }} |
            Категория: {{ post.category.name }}
        </small>
    </div>
{% empty %}
    <p>Ничего не найдено.</p>
{% endfor %}

<div style="margin-top: 16px;">
    {% if page_obj.has_previous %}
        <a href="?{% if q %}q={{ q }}&{% endif %}page={{ page_obj.previous_page_number }}">← Назад</a>
    {% endif %}

    <span style="margin: 0 10px;">
        Страница {{ page_obj.number }} из {{ page_obj.paginator.num_pages }}
    </span>

    {% if page_obj.has_next %}
        <a href="?{% if q %}q={{ q }}&{% endif %}page={{ page_obj.next_page_number }}">Вперёд →</a>
    {% endif %}
</div>
{% endblock %}
""", encoding="utf-8")

(tpl_dir / "category_posts.html").write_text("""\
{% extends 'blog/base.html' %}

{% block title %}Категория: {{ category.name }}{% endblock %}

{% block content %}
<h2>Категория: {{ category.name }}</h2>
<p>{{ category.description }}</p>

{% if q %}
    <p>Результаты по запросу: <strong>{{ q }}</strong></p>
{% endif %}

{% for post in page_obj %}
    <div class="post">
        <h3><a href="{% url 'post_detail' pk=post.pk %}">{{ post.title }}</a></h3>
        <p>{{ post.content|truncatewords:30 }}</p>
        <small>Опубликовано: {{ post.published_date|date:"d.m.Y H:i" }}</small>
    </div>
{% empty %}
    <p>Ничего не найдено.</p>
{% endfor %}

<div style="margin-top: 16px;">
    {% if page_obj.has_previous %}
        <a href="?{% if q %}q={{ q }}&{% endif %}page={{ page_obj.previous_page_number }}">← Назад</a>
    {% endif %}

    <span style="margin: 0 10px;">
        Страница {{ page_obj.number }} из {{ page_obj.paginator.num_pages }}
    </span>

    {% if page_obj.has_next %}
        <a href="?{% if q %}q={{ q }}&{% endif %}page={{ page_obj.next_page_number }}">Вперёд →</a>
    {% endif %}
</div>
{% endblock %}
""", encoding="utf-8")

print("OK: pagination added")


OK: pagination added


In [53]:
PROJECT_DIR = Path("myblog").resolve()

if server_proc and server_proc.poll() is None:
    server_proc.terminate()

server_proc = subprocess.Popen(
    [sys.executable, "manage.py", "runserver", "--noreload", "127.0.0.1:8001"],
    cwd=str(PROJECT_DIR),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

print("OK: сервер перезапущен -> http://127.0.0.1:8001/")


OK: сервер перезапущен -> http://127.0.0.1:8001/


In [57]:
PROJECT_DIR = Path("myblog").resolve()

username = "admin"
password = "admin12345"
email = "admin@yandex.ru"

script = f"""
from django.contrib.auth import get_user_model
User = get_user_model()

username = {username!r}
password = {password!r}
email = {email!r}

u = User.objects.filter(username=username).first()
if u:
    u.is_staff = True
    u.is_superuser = True
    u.set_password(password)
    if hasattr(u, "email"):
        u.email = email
    u.save()
    print("OK: admin обновлён ->", username)
else:
    User.objects.create_superuser(username=username, email=email, password=password)
    print("OK: admin создан ->", username)
"""

subprocess.check_call([sys.executable, "manage.py", "shell", "-c", script], cwd=str(PROJECT_DIR))

print("login:", username)
print("password:", password)
print("url: http://127.0.0.1:8001/admin/")


15 objects imported automatically (use -v 2 for details).

OK: admin обновлён -> admin
login: admin
password: admin12345
url: http://127.0.0.1:8001/admin/
